In [2]:
import sys
import warnings
sys.path.append("../Pythonscripts")
sys.path.append(r"C:\Users\atedi\OneDrive\Documents\SageBionetworks\synapse\Scripts")
import pandas as pd
import numpy as np
import synapseclient as sc
warnings.simplefilter("ignore")

In [3]:
## login to synapse
syn = sc.login()

Welcome, aryton tediarjo!



In [7]:
import sys
import warnings
import pandas as pd
import numpy as np
import synapseclient as sc
warnings.simplefilter("ignore")

DEMOGRAPHIC_TABLE_V1 = "syn8381056"
DEMOGRAPHIC_TABLE_V2 = "syn15673379"
WALKING_TABLE_V1     = "syn21021435"
WALKING_TABLE_V2     = "syn21018127"
BALANCE_TABLE_V1     = "syn21022189"
BALANCE_TABLE_V2     = "syn21018245"


def clean_data(version, demographic_table, table):
    if version == "V1":
        ## demographic data ## 
        demographic_entity = syn.get(demographic_table)
        demographic_data   = pd.read_csv(demographic_entity["path"], sep = "\t")[["healthCode", "PD"]]
    else:
        demographic_data   = syn.tableQuery("SELECT distinct(healthCode) as healthCode, \
                                             diagnosis as PD from {}".format(demographic_table))
        demographic_data   = demographic_data.asDataFrame().drop_duplicates("healthCode", keep = "first").reset_index(drop = True)
        demographic_data   = demographic_data[demographic_data["PD"] != "no_answer"] 
    demographic_data["PD"] = demographic_data["PD"].map({"parkinsons":1, "control":0})
    entity = syn.get(table)
    data   = pd.read_csv(entity["path"], index_col = 0)
    data = (data[data["phoneInfo"].str.contains("iPhone")]) \
                            [(data != "#ERROR").all(axis = 1)]
    data.drop_duplicates(subset=['healthCode', 'createdOn'], keep = "first", inplace = True)
    data[[_ for _ in data.columns if "feat" in _]] = \
    data[[_ for _ in data.columns if "feat" in _]].apply(pd.to_numeric)
    
    data.reset_index(drop = True, inplace = True)
    data = pd.merge(data, demographic_data, 
                    how = "inner", on = "healthCode")
    return data
    

def main():
    cleaned_PDKIT_MPV1 = clean_data("V1", DEMOGRAPHIC_TABLE_V1, WALKING_TABLE_V1)
    cleaned_PDKIT_MPV2 = clean_data("V2", DEMOGRAPHIC_TABLE_V2, WALKING_TABLE_V2)
    cleaned_SFM_MPV1 = clean_data("V1", DEMOGRAPHIC_TABLE_V2, BALANCE_TABLE_V1)
    cleaned_SFM_MPV2 = clean_data("V2", DEMOGRAPHIC_TABLE_V2, BALANCE_TABLE_V2)

In [10]:
cleaned_PDKIT_MPV1 = clean_data("V1", DEMOGRAPHIC_TABLE_V1, WALKING_TABLE_V1)
cleaned_PDKIT_MPV2 = clean_data("V2", DEMOGRAPHIC_TABLE_V2, WALKING_TABLE_V2)
cleaned_SFM_MPV1 = clean_data("V1", DEMOGRAPHIC_TABLE_V1, BALANCE_TABLE_V1)
cleaned_SFM_MPV2 = clean_data("V2", DEMOGRAPHIC_TABLE_V2, BALANCE_TABLE_V2)

In [ ]:
CLEANED_pdkit_mp